# **Effective Model**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from MTI_Second_Order import FMTI2_NeumannBC, Change_Basis, Block_Reverse

# labels for curves
Flabels = [["$f_{\\uparrow +, \\uparrow +}$", "$f_{\\uparrow +, \downarrow +}$", "$f_{\\uparrow +, \\uparrow -}$", "$f_{\\uparrow +, \downarrow -}$"],
          ["$f_{\downarrow +, \\uparrow +}$", "$f_{\downarrow +, \downarrow +}$", "$f_{\downarrow +, \\uparrow -}$", "$f_{\downarrow +, \downarrow -}$"],
          ["$f_{\\uparrow -, \\uparrow +}$", "$f_{\\uparrow -, \downarrow +}$", "$f_{\\uparrow -, \\uparrow -}$", "$f_{\\uparrow -, \downarrow -}$"],
          ["$f_{\downarrow -, \\uparrow +}$", "$f_{\downarrow -, \downarrow +}$", "$f_{\downarrow -, \\uparrow -}$", "$f_{\downarrow -, \downarrow -}$"]]

# colors for plotting
colors = ["#0077FF", "#00CC00", "#FFD700", "#FF2200"]

## **Channel Decomposition** 

We decompose the averaged induced pairing $\overline{\Delta}_{ind} (z)$ over a basis of 16 matrices that generates the whole space made by the outer product of spin $\sigma = \uparrow, \downarrow$ and orbital $\lambda=\pm$.
The $4 \times 4$ matrix can be decomposed as
$$
    \overline{\Delta}_{ind} (z) = \sum_A f_A (z) \Lambda_A \,,
$$
where the coefficients of the expansion can be computed through
$$
    f_A (z) = 
    \frac{ \text{Tr}\left( \Lambda_A^\dagger \, \Delta_{ind} (z)  \right) }
    { \text{Tr}\left( \Lambda_A^\dagger \, \Lambda_A  \right) }
$$and $\{ \Lambda_A \}$ is a basis of 16 matrices that generates the whole space.


For the sake of simplicity, we choose as basis in spin and orbital spaces the set of Pauli matrices 
$$
    S_a = 
    \left\{ 
    \sigma_0, \sigma_x, \sigma_y, \sigma_z 
    \right\} \,,
    \qquad
    T_a = 
    \left\{ 
    \tau_0, \tau_x, \tau_y, \tau_z 
    \right\} \,,   
$$
so that the full set of matrices $\{ \Lambda_A \}$ can be simply obtained as the outer product
$$
    \Lambda_A = \frac{1}{2} \left( S_a \otimes L_\alpha \right) \,,
$$
where the factor $\frac12$ is used to normalize the Hilbert-Schmidt inner product of the matrices $\Lambda_A$.
Here $a \in \{ \sigma_0, \sigma_x, \sigma_y, \sigma_z \}$ and $\alpha \in \{ \tau_0, \tau_x, \tau_y, \tau_z \}$ denote the matrices of the spin and orbital subspaces.


#### <font color='green'> **Pairing Channels** </font> 

In [9]:
from IPython.display import Math, display
from sympy import Matrix, Symbol, latex, simplify, nsimplify, sqrt
from Pairing_Effective_Model import Simpy_Projection, Render_Projection, Render_Channel, Pairing_Projection, Pairing_Channel, Project_All, Reconstruct

# define your physical basis labels (spin x orbital)
basis = [r"\uparrow+", r"\uparrow-", r"\downarrow+", r"\downarrow-"]

# build a 4×4 Matrix of Symbols f_{α,β}
M = Matrix([[ Symbol(f"f_{{{i},{j}}}") for j in basis ] for i in basis ])

# spin and ordital indices
s = 'asym'; o = '++'

# projection 
Render_Projection(M=M, spin=s, orbital=o)
print()
# pairing channel
Render_Channel(spin=s, orbital=o)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### <font color='green'> **Singlet-Triplet Decomposition** </font> 

In [12]:
# build a 4×4 Matrix of Symbols f_{α,β}
F = Matrix([[ Symbol(f"f_{{{i},{j}}}") for j in basis ] for i in basis ])

# swap spin only
F_swap = Matrix(Block_Reverse(F))

# matrices for signlet
F_odd = 1/2*(F - F_swap)
# matrices for triplet
F_even = 1/2*(F + F_swap)

# spin singlet S=0 part
Fs_0 = (sqrt(2)*F_odd[0:2, 2:4]).applyfunc(simplify); fs_0 = (Fs_0.norm())**2

# spin triplet S=0 part
Ft_0 = (sqrt(2)*F_even[0:2, 2:4]).applyfunc(simplify); ft_0 = (Ft_0.norm())**2

# spin triplet S=1 part
Ft_p1 = (F_even[0:2, 0:2]).applyfunc(simplify); ft_p1 = (Ft_p1.norm())**2

# spin triplet S=-1 part
Ft_m1 = (F_even[2:4, 2:4]).applyfunc(simplify); ft_m1 = (Ft_m1.norm())**2


print('Singlet S=0')
display(Matrix(Fs_0.applyfunc(nsimplify)))
display(nsimplify(fs_0))
print()

print('Triplet S=0')
display(Matrix(Ft_0.applyfunc(nsimplify)))
display(nsimplify(ft_0))
print()

print('Triplet S=+1')
display(Matrix(Ft_p1.applyfunc(nsimplify)))
display(nsimplify(ft_p1))
print()

print('Triplet S=-1')
display(Matrix(Ft_m1.applyfunc(nsimplify)))
display(nsimplify(ft_m1))

Singlet S=0


⎡√2⋅(-f_{\downarrow+,\uparrow+} + f_{\uparrow+,\downarrow+})  √2⋅(-f_{\downarrow+,\uparrow-} + f_{\uparrow+,\downarrow-})⎤
⎢───────────────────────────────────────────────────────────  ───────────────────────────────────────────────────────────⎥
⎢                             2                                                            2                             ⎥
⎢                                                                                                                        ⎥
⎢√2⋅(-f_{\downarrow-,\uparrow+} + f_{\uparrow-,\downarrow+})  √2⋅(-f_{\downarrow-,\uparrow-} + f_{\uparrow-,\downarrow-})⎥
⎢───────────────────────────────────────────────────────────  ───────────────────────────────────────────────────────────⎥
⎣                             2                                                            2                             ⎦

                                                       2                                                          2                                         
│f_{\downarrow+,\uparrow+} - f_{\uparrow+,\downarrow+}│    │f_{\downarrow+,\uparrow-} - f_{\uparrow+,\downarrow-}│    │f_{\downarrow-,\uparrow+} - f_{\uparr
──────────────────────────────────────────────────────── + ──────────────────────────────────────────────────────── + ──────────────────────────────────────
                           2                                                          2                                                          2          

                 2                                                          2
ow-,\downarrow+}│    │f_{\downarrow-,\uparrow-} - f_{\uparrow-,\downarrow-}│ 
────────────────── + ────────────────────────────────────────────────────────
                                                2                            


Triplet S=0


⎡√2⋅(f_{\downarrow+,\uparrow+} + f_{\uparrow+,\downarrow+})  √2⋅(f_{\downarrow+,\uparrow-} + f_{\uparrow+,\downarrow-})⎤
⎢──────────────────────────────────────────────────────────  ──────────────────────────────────────────────────────────⎥
⎢                            2                                                           2                             ⎥
⎢                                                                                                                      ⎥
⎢√2⋅(f_{\downarrow-,\uparrow+} + f_{\uparrow-,\downarrow+})  √2⋅(f_{\downarrow-,\uparrow-} + f_{\uparrow-,\downarrow-})⎥
⎢──────────────────────────────────────────────────────────  ──────────────────────────────────────────────────────────⎥
⎣                            2                                                           2                             ⎦

...

⎡f_{\uparrow+,\uparrow+}  f_{\uparrow+,\uparrow-}⎤
⎢                                                ⎥
⎣f_{\uparrow-,\uparrow+}  f_{\uparrow-,\uparrow-}⎦

                         2                            2                            2                            2
│f_{\uparrow+,\uparrow+}│  + │f_{\uparrow+,\uparrow-}│  + │f_{\uparrow-,\uparrow+}│  + │f_{\uparrow-,\uparrow-}│ 


Triplet S=-1


⎡f_{\downarrow+,\downarrow+}  f_{\downarrow+,\downarrow-}⎤
⎢                                                        ⎥
⎣f_{\downarrow-,\downarrow+}  f_{\downarrow-,\downarrow-}⎦

                             2                                2                                2                                2
│f_{\downarrow+,\downarrow+}│  + │f_{\downarrow+,\downarrow-}│  + │f_{\downarrow-,\downarrow+}│  + │f_{\downarrow-,\downarrow-}│ 

#### <font color='green'> **Singlet-Triplet Projection** </font> 

In [13]:
# build a 4×4 Matrix of Symbols f_{α,β}
F = Matrix([[ Symbol(f"f_{{{i},{j}}}") for j in basis ] for i in basis ])

spin = {
    'uu': 'Triplet S=+1',  
    'dd': 'Triplet S=-1',
    'sym': 'Triplet S=0',
    'asym': 'Singlet S=0'
}

fcomp = {
    'uu': ft_p1,  
    'dd': ft_m1,
    'sym': ft_0,
    'asym': fs_0
}

# loop over spin 
for s in ['asym', 'sym','uu', 'dd']:

    f = 0
    # loop over orbitals 
    for o in ['++', '+-', '-+', '--']:

        # evaluate projection
        proj = Simpy_Projection(M=F, spin=s, orbital=o)
        f += abs(proj)**2

    # display projection
    print(spin[s])
    print()
    print('Projection:')
    display(f)
    print('Matrix Decomposition:')
    # display norm of submatrices
    display(nsimplify(fcomp[s]))
    print()

Singlet S=0

Projection:


                                                       2                                                          2                                         
│f_{\downarrow+,\uparrow+} - f_{\uparrow+,\downarrow+}│    │f_{\downarrow+,\uparrow-} - f_{\uparrow+,\downarrow-}│    │f_{\downarrow-,\uparrow+} - f_{\uparr
──────────────────────────────────────────────────────── + ──────────────────────────────────────────────────────── + ──────────────────────────────────────
                           2                                                          2                                                          2          

                 2                                                          2
ow-,\downarrow+}│    │f_{\downarrow-,\uparrow-} - f_{\uparrow-,\downarrow-}│ 
────────────────── + ────────────────────────────────────────────────────────
                                                2                            

Matrix Decomposition:


                                                       2                                                          2                                         
│f_{\downarrow+,\uparrow+} - f_{\uparrow+,\downarrow+}│    │f_{\downarrow+,\uparrow-} - f_{\uparrow+,\downarrow-}│    │f_{\downarrow-,\uparrow+} - f_{\uparr
──────────────────────────────────────────────────────── + ──────────────────────────────────────────────────────── + ──────────────────────────────────────
                           2                                                          2                                                          2          

                 2                                                          2
ow-,\downarrow+}│    │f_{\downarrow-,\uparrow-} - f_{\uparrow-,\downarrow-}│ 
────────────────── + ────────────────────────────────────────────────────────
                                                2                            


Triplet S=0

Projection:


...

                         2                            2                            2                            2
│f_{\uparrow+,\uparrow+}│  + │f_{\uparrow+,\uparrow-}│  + │f_{\uparrow-,\uparrow+}│  + │f_{\uparrow-,\uparrow-}│ 


Triplet S=-1

Projection:


                             2                                2                                2                                2
│f_{\downarrow+,\downarrow+}│  + │f_{\downarrow+,\downarrow-}│  + │f_{\downarrow-,\downarrow+}│  + │f_{\downarrow-,\downarrow-}│ 

Matrix Decomposition:


                             2                                2                                2                                2
│f_{\downarrow+,\downarrow+}│  + │f_{\downarrow+,\downarrow-}│  + │f_{\downarrow-,\downarrow+}│  + │f_{\downarrow-,\downarrow-}│ 

### **Induced Pairing**

In [5]:
# shape of the matrix 
n = 4

# space dimensions
K = []; Z = []
k_dim = 100; z_dim = 100

# read file pairing
rows = np.loadtxt("Pairing/pairing_L-0.txt")

# store data in matrices
F_data = np.empty((k_dim, z_dim, 4, 4), dtype='complex')


# loop over momenta
for idx in range(k_dim):
    
    # loop over space
    for jdx in range(z_dim):

        # index for rows
        irow = z_dim * idx + jdx
                
        # read one row
        row = rows[irow]
    
        if jdx == 0: K.append(float(row[0]))
    
        if idx == 0: Z.append(float(row[1]))
    
        # real part
        real = row[2 : 2 + n*n].reshape((n, n))
        # imaginary part
        imag = row[2 + n*n :].reshape((n, n))
        # pairing matrix
        matrix = real + 1j * imag

        F_data[idx][jdx] = Change_Basis(matrix)


# take sum over k
Favg = np.sum(F_data, axis=0)
print(Favg.shape)

(100, 4, 4)


#### <font color='green'> **Singlet-Triplet Decomposition** </font> 

In [16]:
# swap spin only
F_swap = np.array([[Block_Reverse(f) for f in F1] for F1 in F_data])

# matrices for signlet
F_odd = 0.5*(F_data - F_swap)
# matrices for triplet
F_even = 0.5*(F_data + F_swap)


# spin singlet S=0 part
Fs_0 = np.sqrt(2)*F_odd[:, :, 0:2, 2:4]; fs_0 = np.linalg.norm(Fs_0, axis=(2, 3))

# spin triplet S=0 part
Ft_0 = np.sqrt(2)*F_even[:, :, 0:2, 2:4]; ft_0 = np.linalg.norm(Ft_0, axis=(2, 3))
# spin triplet S=1 part
Ft_p1 = F_even[:, :, 0:2, 0:2]; ft_p1 = np.linalg.norm(Ft_p1, axis=(2, 3))
# spin triplet S=-1 part
Ft_m1 = F_even[:, :, 2:4, 2:4]; ft_m1 = np.linalg.norm(Ft_m1, axis=(2, 3))


norm = np.sum(ft_0+ft_p1+ft_m1+fs_0, axis=0)[0]

for f,s in zip([fs_0, ft_0, ft_p1, ft_m1], ['asym', 'sym', 'uu', 'dd']):

    print(spin[s])
    print( (np.sum(f, axis=0)/norm)[0] )
    print()

#print(ft_0)

Singlet S=0
0.2522973628933872

Triplet S=0
0.24806908451687745

Triplet S=+1
0.2498167762948679

Triplet S=-1
0.2498167762948679



#### <font color='green'> **Singlet-Triplet Projection** </font> 

In [18]:
F0 = Favg[0]

fproj = np.zeros(4)
# loop over spin 
for fdec,s,idx in zip([fs_0, ft_0, ft_p1, ft_m1], ['asym', 'sym', 'uu', 'dd'], range(4)):

    # display projection
    print(spin[s])
    print()
    
    # loop over orbitals 
    for o in ['++', '+-', '-+', '--']:

        # evaluate projection
        proj = Pairing_Projection(Delta=F0, spin=s, orbital=o)
        # sum projections with same spin 
        fproj[idx] += abs(proj)**2

        print(abs(proj)**2)
        
    fproj[idx] = np.sqrt(fproj[idx])
    
    
    print('Projection:')
    print(fproj[idx])
    
    print('Matrix Decomposition:')
    print( (np.sum(fdec, axis=0)/norm)[0] )
    print()

Singlet S=0

1.2137493878006536e+18
498816079664.6414
726775929754.4044
7.194656557530052e+17
Projection:
1390401477.6839342
Matrix Decomposition:
0.2522973628933872

Triplet S=0

890749693013.7712
9.344790087277975e+17
9.344791889660596e+17
411018803128.88824
Projection:
1367098935.5062616
Matrix Decomposition:
0.24806908451687745

Triplet S=+1

1.4169257138920584e-16
1.4941893544599924e-16
1.3962837277626243e-16
5.653877689682267e-16
Projection:
3.1561489961338866e-08
Matrix Decomposition:
0.2498167762948679

Triplet S=-1

1.4169257138920584e-16
1.4941893544599924e-16
1.3962837277626243e-16
5.653877689682267e-16
Projection:
3.1561489961338866e-08
Matrix Decomposition:
0.2498167762948679

